In [156]:
import geopandas as gpd
import numpy as np
import pandas as pd

## Open County Data

In [157]:
file_path = "data/cb_2018_us_county_500k/cb_2018_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)

In [158]:
counties_gdf = counties_gdf.rename(columns={"GEOID": "FIPS"})

## Open 2023 Population Estimates

In [159]:
county_populations = pd.read_csv(
    "data/co-est2023-alldata.csv", sep=",", encoding="latin-1"
)

In [160]:
county_populations["FIPS"] = county_populations.apply(
    lambda row: f"{row['STATE']:02d}{row['COUNTY']:03d}", axis=1
)

In [161]:
county_populations_2023_df = county_populations[["POPESTIMATE2023", "FIPS"]].dropna()

## Merge data

In [162]:
county_populations_2023_gdf = counties_gdf.merge(
    county_populations_2023_df, on="FIPS", how="left"
)

In [163]:
county_populations_2023_gdf = county_populations_2023_gdf[
    county_populations_2023_gdf["STATEFP"] == "48"
]

## Get Whataburger Data

In [164]:
whataburger_gdf = gpd.read_file("data/whataburger_locations.gpkg")

## Intersect Data

In [165]:
county_populations_2023_gdf = county_populations_2023_gdf.to_crs(
    whataburger_gdf.crs.srs
)

In [166]:
counties_count_gdf = county_populations_2023_gdf.join(
    gpd.sjoin(whataburger_gdf, county_populations_2023_gdf)
    .groupby("index_right")
    .size()
    .rename("stores"),
    how="left",
)

In [167]:
counties_count_gdf = counties_count_gdf.fillna(0)
counties_count_gdf["POPESTIMATE2023"] = counties_count_gdf["POPESTIMATE2023"].astype(
    int
)
counties_count_gdf["stores"] = counties_count_gdf["stores"].astype(int)

In [168]:
counties_count_gdf = counties_count_gdf[
    ["geometry", "stores", "POPESTIMATE2023", "NAME"]
]

In [169]:
counties_count_gdf["per_capita"] = counties_count_gdf["POPESTIMATE2023"].div(
    counties_count_gdf["stores"]
)
counties_count_gdf["per_100k"] = (counties_count_gdf["POPESTIMATE2023"] / 100_000).div(
    counties_count_gdf["stores"]
)

In [170]:
counties_count_gdf.replace([np.inf, -np.inf], np.nan, inplace=True)
counties_count_gdf = counties_count_gdf.fillna(0)

In [171]:
counties_count_gdf.to_file("data/counties_count.gpkg")